In [44]:
import tkinter as tk
from tkinter import ttk
from openpyxl import load_workbook

def atualizar_arquivo(candidato):
    arquivo_caminho = 'Urna.xlsx'
    workbook = load_workbook(filename=arquivo_caminho)
    nome_panilha = "Dados"

    sheet = workbook[nome_panilha]

    novo_candidato = True

    for row in sheet.iter_rows(min_row=2):
        candidato_planilha = row[0].value
        votos = row[1].value

        if candidato_planilha == candidato:
            celeula_votos = sheet.cell(row=row[0].row,column=2)
            celeula_votos.value = votos + 1 if votos else 1

            novo_candidato = False
            break

    if novo_candidato:
        nova_linha = (candidato,0)
        sheet.append(nova_linha)
    
    workbook.save(arquivo_caminho)


def contabilizar_votos():
    arquivo_caminho = 'Urna.xlsx'
    workbook = load_workbook(filename=arquivo_caminho)
    nome_panilha = "Dados"

    sheet = workbook[nome_panilha]

    contagem = {}

    for row in sheet.iter_rows(min_row=2,values_only=True):
        candidato = row[0]
        votos = row[1]

        if candidato:
            contagem[candidato] = votos
    
    return contagem

def adicionar_candidato():
    candidato = entrada_texto.get()

    entrada_texto.delete(0,tk.END)

    lista_opcoes.insert(tk.END,candidato)

    exibir_resultado(contabilizar_votos())

    atualizar_arquivo(candidato)

    resultado_text.insert(tk.END, f"Candidato {candidato}: 0 - Voto(s)\n")

    lista_opcoes.delete(0,tk.END)
    carregar_dados()

def votar():
    candidato = entrada_texto.get()

    entrada_texto.delete(0,tk.END)

    if candidato in lista_opcoes.get(0,tk.END):
        atualizar_arquivo(candidato)
        exibir_resultado(contabilizar_votos())        

def remover_candidato():
    candidato = entrada_texto.get()

    entrada_texto.delete(0,tk.END)

    if candidato in lista_opcoes.get(0,tk.END):
        indice = lista_opcoes.get(0,tk.END).index(candidato)

        lista_opcoes.delete(indice)

        exibir_resultado(contabilizar_votos())

        remover_do_arquivo(candidato)

        remover_do_resultado_text(candidato)

def remover_do_resultado_text(candidato):
    linhas_resultado = resultado_text.get(1.0,tk.END).split('\n')
    nova_lina_resultado = ""
    for linha in linhas_resultado:
        if linha.startswith("Candidato {}:".format(candidato)):
            continue
        nova_lina_resultado += linha + "\n"

    resultado_text.delete(1.0,tk.END)
    resultado_text.insert(tk.END, nova_lina_resultado)


def remover_do_arquivo(candidato):
    arquivo_caminho = 'Urna.xlsx'
    workbook = load_workbook(filename=arquivo_caminho)
    nome_panilha = "Dados"

    sheet = workbook[nome_panilha]

    for row in sheet.iter_rows(min_row=2):
        candidato_planilha = row[0].value

        if candidato_planilha == candidato:
            sheet.delete_rows(row[0].row)
            break
    
    workbook.save(arquivo_caminho)


def exibir_resultado(contagem):
    resultado_text.delete(1.0,tk.END)

    resultado_text.insert(tk.END, "Resultado da Votação:\n")

    for candidato, votos in contagem.items():
        resultado_text.insert(tk.END, f"Candidato {candidato}: {votos} - Voto(s)\n")

def carregar_dados():
    arquivo_caminho = 'Urna.xlsx'
    workbook = load_workbook(filename=arquivo_caminho)
    nome_panilha = "Dados"

    sheet = workbook[nome_panilha]

    contagem = {}

    for row in sheet.iter_rows(min_row=2,values_only=True):
        candidato = row[0]
        votos = row[1]

        if candidato:
            contagem[candidato] = votos

            lista_opcoes.insert(tk.END,candidato)

    exibir_resultado(contagem)

def verificar_vencedor():
    contagem = contabilizar_votos()
    votos_maximo = max(contagem.values())
    vencedor = [candidato for candidato, votos, in contagem.items() if votos == votos_maximo]

    if len(vencedor) == 1:
        mensagem = "O vencedor é o candidato {} com {} voto(s)".format(vencedor[0],votos_maximo)
    else:
        mensagem = "Houve empate entre os seguintes candidatos {} com {} voto(s)".format(", ".join(vencedor),votos_maximo)

    resultado_text.delete(1.0,tk.END)
    resultado_text.insert(tk.END,mensagem)
    

tela_principal = tk.Tk()
tela_principal.title("Urna Eletrônica")

estilo = ttk.Style()
estilo.configure("TButton", font=("Arial",12),padding=10)
estilo.configure("TLabel", font=("Arial",12),padding=10)

frame_principal = ttk.Frame(tela_principal,padding=20)
frame_principal.grid(row=0, column=0, sticky='NSEW')

frame_esquerda = ttk.Frame(tela_principal,padding=20)
frame_esquerda.grid(row=0, column=0, sticky='NSEW')

frame_esquerda.columnconfigure(0,weight=1)
frame_esquerda.rowconfigure(0,weight=1)

lista_opcoes = tk.Listbox(frame_esquerda,font=("Arial",12))
lista_opcoes.grid(row=0, column=0, sticky='NSEW')

numero_canditado_label = ttk.Label(frame_esquerda,
                                   text="Digite o núumero do candidato",
                                   font=("Arial 12"))
numero_canditado_label.grid(row=1, column=0, sticky='NSEW')

entrada_texto = tk.Entry(frame_esquerda,
                         font=("Arial 20"))
entrada_texto.grid(row=2, column=0, sticky='NSEW')

adicionar_botao = ttk.Button(frame_esquerda,
                            text="Adicionar Candidato",
                            command=adicionar_candidato)
adicionar_botao.grid(row=3, column=0, sticky='EW')

remover_botao = ttk.Button(frame_esquerda,
                            text="Remover Candidato",
                            command=remover_candidato)
remover_botao.grid(row=4, column=0, sticky='EW')

votar_botao = ttk.Button(frame_esquerda,
                            text="Votar",
                            command=votar)
votar_botao.grid(row=5, column=0, sticky='EW')

verificar_botao = ttk.Button(frame_esquerda,
                            text="Verificar Vencedor",
                            command=verificar_vencedor)
verificar_botao.grid(row=6, column=0, sticky='EW')

frame_direita = ttk.Frame(tela_principal)
frame_direita.grid(row=0, column=1, sticky='NSEW')

resultado_label = ttk.Label(frame_direita,
                            text="Resultado da Votação",
                            font=("Arial 16"))
resultado_label.pack()

resultado_text = tk.Text(frame_direita,
                            font=("Arial 16"))
resultado_text.pack(fill="both",expand=True,padx=10,pady=10)

tela_principal.columnconfigure(0,weight=1)
tela_principal.rowconfigure(0,weight=1)

frame_direita.columnconfigure(0,weight=1)
frame_direita.rowconfigure(0,weight=1)

carregar_dados()

tela_principal.mainloop()